In [48]:
! git clone https://github.com/uf-hobi-informatics-lab/ClinicalTransformerNER.git

Cloning into 'ClinicalTransformerNER'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 92 (delta 37), reused 73 (delta 25), pack-reused 0
Unpacking objects: 100% (92/92), done.


In [2]:
import sys
sys.path.append("./ClinicalTransformerNER/")
sys.path.append("./NLPpreprocessing-master/")
import os
from pathlib import Path
from collections import defaultdict, Counter
import numpy as np
from sklearn.model_selection import train_test_split
import shutil 
from annotation2BIO import generate_BIO, pre_processing, read_annotation_brat, BIOdata_to_file
MIMICIII_PATTERN = "\[\*\*|\*\*\]"

In [3]:
! ls


bio			new_round_merged
bio_final		new_round_merged_1_2
ClinicalTransformerNER	NLPpreprocessing-master
data			round_3_output
log_bert.txt		round_3_output_formatted_output
log.txt			round_4_output_formatted_output
model			SDOH_bert
new_round_3		SDoH_ner-Copy1.ipynb
new_round_4		SDoH_ner.ipynb


In [3]:
! pwd

/home/zehao.yu/workspace/py3/SDoH


In [62]:
all_root='./new_round_merged/'

In [4]:
training_root='./new_round_merged/'

In [8]:
#split train and test annotation
file_ids = list(file_ids)
train_dev_ids, test_ids = train_test_split(file_ids, train_size=0.8, random_state=13, shuffle=True)
len(train_dev_ids), len(test_ids)
train_dev_root = Path('./data/used/train_set')
test_root = Path('./data/used/test_set')
#create notes file
Path(train_dev_root).mkdir(parents=True, exist_ok=True)
Path(test_root).mkdir(parents=True, exist_ok=True)
train_root=Path(training_root)
#copy file to train and test
for fid in train_dev_ids:
    txt_fn = train_root / (fid + ".txt")
    ann_fn = train_root / (fid + ".ann")
    txt_fn1 = train_dev_root / (fid + ".txt")
    ann_fn1 = train_dev_root / (fid + ".ann")
    shutil.copyfile(txt_fn, txt_fn1)
    shutil.copyfile(ann_fn, ann_fn1)
for fid in test_ids:
    txt_fn = train_root / (fid + ".txt")
    ann_fn = train_root / (fid + ".ann")
    txt_fn1 = test_root / (fid + ".txt")
    ann_fn1 = test_root / (fid + ".ann")
    shutil.copyfile(txt_fn, txt_fn1)
    shutil.copyfile(ann_fn, ann_fn1)

In [9]:
# Brief descriptive statistics
file_ids = set()
enss = []

for fn in train_dev_root.glob("*.ann"):
    file_ids.add(fn.stem)
    _, ens, _ = read_annotation_brat(fn)
    #print( _)
    enss.extend(ens)
print("training files: ", len(file_ids), list(file_ids)[:5])
print("total training eneitites: ", len(enss))
print("Entities distribution by types:\n", "\n".join([str(c) for c in Counter([each[1] for each in enss]).most_common()]))

training files:  503 ['note_162059_0', 'note_143983_0', 'IRB201901754_NOTE_340407_0', 'IRB201901754_NOTE_661097_0', 'IRB201901754_NOTE_1017291_0']
total training eneitites:  10506
Entities distribution by types:
 ('Substance_use_status', 1825)
('Sdoh_status', 1241)
('Gender', 679)
('Smoking_type', 579)
('Education', 483)
('Alcohol_use', 458)
('Living_supply', 445)
('Drug_use', 404)
('Marital_status', 402)
('Occupation', 368)
('Sex_act', 368)
('Abuse', 331)
('Social_cohesion', 329)
('Smoking_freq_ppd', 313)
('Tobacco_use', 273)
('Smoking_freq_sy', 223)
('Smoking_freq_qy', 201)
('Physical_act', 179)
('Alcohol_freq', 173)
('Sdoh_freq', 159)
('Transportation', 154)
('Partner', 134)
('Living_Condition', 101)
('Race', 95)
('Drug_type', 81)
('Financial_constrain', 78)
('Smoking_freq_py', 62)
('Alcohol_other', 54)
('SDoH_ICD', 50)
('Alcohol_type', 44)
('Smoking_freq_other', 40)
('Protection', 38)
('Employment_status', 33)
('Ethnicity', 28)
('Gender_status', 24)
('Language', 21)
('Employment_lo

In [28]:
lenall=0
for fn in train_dev_root.glob("*.ann"):
    
    #print(fn)
    with open(fn,'r') as f:
        llines=f.read()
        len1=len(llines.split('\t'))
        #print(len1)
        f.close()
    lenall=lenall+len1
    #file_ids.add(fn.stem)

In [29]:
lenall

29058

In [10]:
# generate BIO from brat annotation
train_root = train_dev_root
train_bio = "./bio_final/trains/"
output_root = Path(train_bio)
output_root.mkdir(parents=True, exist_ok=True)

for fid in file_ids:
    txt_fn = train_root / (fid + ".txt")
    ann_fn = train_root / (fid + ".ann")
    bio_fn = output_root / (fid + ".bio.txt")
    
    txt, sents = pre_processing(txt_fn, deid_pattern=MIMICIII_PATTERN)
    e2idx, entities, rels = read_annotation_brat(ann_fn)
    nsents, sent_bound = generate_BIO(sents, entities, file_id=fid, no_overlap=False)
    #print(nsents)
    BIOdata_to_file(bio_fn, nsents)

2021-05-16 21:13:36,590 INFO sentence boundary detection class initiated.
2021-05-16 21:13:36,591 INFO word level tokenization with replace_number set to False
2021-05-16 21:13:36,591 INFO current processing data/used/train_set/note_162059_0.txt ...
2021-05-16 21:13:36,612 INFO process note_162059_0 file
2021-05-16 21:13:36,661 INFO sentence boundary detection class initiated.
2021-05-16 21:13:36,662 INFO word level tokenization with replace_number set to False
2021-05-16 21:13:36,663 INFO current processing data/used/train_set/note_143983_0.txt ...
2021-05-16 21:13:36,682 INFO process note_143983_0 file
2021-05-16 21:13:36,729 INFO sentence boundary detection class initiated.
2021-05-16 21:13:36,730 INFO word level tokenization with replace_number set to False
2021-05-16 21:13:36,731 INFO current processing data/used/train_set/IRB201901754_NOTE_340407_0.txt ...
2021-05-16 21:13:36,751 INFO process IRB201901754_NOTE_340407_0 file
2021-05-16 21:13:36,799 INFO sentence boundary detection

2021-05-16 21:13:38,614 INFO current processing data/used/train_set/note_237892_1.txt ...
2021-05-16 21:13:38,633 INFO process note_237892_1 file
2021-05-16 21:13:38,681 INFO sentence boundary detection class initiated.
2021-05-16 21:13:38,682 INFO word level tokenization with replace_number set to False
2021-05-16 21:13:38,682 INFO current processing data/used/train_set/IRB201901754_NOTE_1188005_0.txt ...
2021-05-16 21:13:38,703 INFO process IRB201901754_NOTE_1188005_0 file
2021-05-16 21:13:38,753 INFO sentence boundary detection class initiated.
2021-05-16 21:13:38,754 INFO word level tokenization with replace_number set to False
2021-05-16 21:13:38,754 INFO current processing data/used/train_set/IRB201901754_NOTE_832658_0.txt ...
2021-05-16 21:13:38,791 INFO process IRB201901754_NOTE_832658_0 file
2021-05-16 21:13:38,843 INFO sentence boundary detection class initiated.
2021-05-16 21:13:38,844 INFO word level tokenization with replace_number set to False
2021-05-16 21:13:38,844 INFO

2021-05-16 21:13:40,422 INFO current processing data/used/train_set/note_285022_0.txt ...
2021-05-16 21:13:40,453 INFO process note_285022_0 file
2021-05-16 21:13:40,503 INFO sentence boundary detection class initiated.
2021-05-16 21:13:40,504 INFO word level tokenization with replace_number set to False
2021-05-16 21:13:40,504 INFO current processing data/used/train_set/IRB201901754_NOTE_352025_3.txt ...
2021-05-16 21:13:40,526 INFO process IRB201901754_NOTE_352025_3 file
2021-05-16 21:13:40,527 WARNING ('[**DATE**]', 'Smoking_freq_qy', (5058, 5068)) offset is overlapped with previous entity; current tok not overlap
2021-05-16 21:13:40,528 WARNING ('[**DATE**]', 'Smoking_freq_qy', (5117, 5127)) offset is overlapped with previous entity; current tok not overlap
2021-05-16 21:13:40,528 WARNING ('beer', 'Alcohol_type', (5172, 5176)) offset is overlapped with previous entity; current tok not overlap
2021-05-16 21:13:40,529 WARNING ('quit in [**DATE**]', 'Alcohol_other', (5199, 5217)) offs

2021-05-16 21:13:42,138 INFO current processing data/used/train_set/note_54640_0.txt ...
2021-05-16 21:13:42,156 INFO process note_54640_0 file
2021-05-16 21:13:42,203 INFO sentence boundary detection class initiated.
2021-05-16 21:13:42,204 INFO word level tokenization with replace_number set to False
2021-05-16 21:13:42,205 INFO current processing data/used/train_set/note_148122_0.txt ...
2021-05-16 21:13:42,216 WARNING '38.9°' cannot be parsed by current rule.
2021-05-16 21:13:42,216 WARNING '38.9°' cannot be parsed by current rule.
2021-05-16 21:13:42,217 WARNING '100.4°' cannot be parsed by current rule.
2021-05-16 21:13:42,221 WARNING '3.1½' cannot be parsed by current rule.
2021-05-16 21:13:42,305 INFO process note_148122_0 file
2021-05-16 21:13:42,371 INFO sentence boundary detection class initiated.
2021-05-16 21:13:42,372 INFO word level tokenization with replace_number set to False
2021-05-16 21:13:42,372 INFO current processing data/used/train_set/note_168943_0.txt ...
2021

2021-05-16 21:13:44,201 WARNING ('[**DATE**]', 'Smoking_freq_qy', (4806, 4816)) offset is overlapped with previous entity; current tok not overlap
2021-05-16 21:13:44,201 WARNING ('beers', 'Alcohol_type', (4984, 4989)) offset is overlapped with previous entity; current tok not overlap
2021-05-16 21:13:44,253 INFO sentence boundary detection class initiated.
2021-05-16 21:13:44,254 INFO word level tokenization with replace_number set to False
2021-05-16 21:13:44,255 INFO current processing data/used/train_set/IRB201901754_NOTE_105944_4.txt ...
2021-05-16 21:13:44,264 WARNING '1.73M2' cannot be parsed by current rule.
2021-05-16 21:13:44,265 WARNING '1.73M2' cannot be parsed by current rule.
2021-05-16 21:13:44,291 INFO process IRB201901754_NOTE_105944_4 file
2021-05-16 21:13:44,345 INFO sentence boundary detection class initiated.
2021-05-16 21:13:44,346 INFO word level tokenization with replace_number set to False
2021-05-16 21:13:44,347 INFO current processing data/used/train_set/IRB2

...

2021-05-16 21:14:09,310 INFO word level tokenization with replace_number set to False
2021-05-16 21:14:09,310 INFO current processing data/used/train_set/IRB201901754_NOTE_1014317_0.txt ...
2021-05-16 21:14:09,333 INFO process IRB201901754_NOTE_1014317_0 file
2021-05-16 21:14:09,381 INFO sentence boundary detection class initiated.
2021-05-16 21:14:09,382 INFO word level tokenization with replace_number set to False
2021-05-16 21:14:09,383 INFO current processing data/used/train_set/note_13473_0.txt ...
2021-05-16 21:14:09,417 INFO process note_13473_0 file
2021-05-16 21:14:09,469 INFO sentence boundary detection class initiated.
2021-05-16 21:14:09,470 INFO word level tokenization with replace_number set to False
2021-05-16 21:14:09,471 INFO current processing data/used/train_set/note_96181_0.txt ...
2021-05-16 21:14:09,495 INFO process note_96181_0 file
2021-05-16 21:14:09,545 INFO sentence boundary detection class initiated.
2021-05-16 21:14:09,546 INFO word level tokenization with 

2021-05-16 21:14:11,129 INFO process note_235572_0 file
2021-05-16 21:14:11,179 INFO sentence boundary detection class initiated.
2021-05-16 21:14:11,180 INFO word level tokenization with replace_number set to False
2021-05-16 21:14:11,180 INFO current processing data/used/train_set/note_53302_1.txt ...
2021-05-16 21:14:11,203 INFO process note_53302_1 file
2021-05-16 21:14:11,252 INFO sentence boundary detection class initiated.
2021-05-16 21:14:11,253 INFO word level tokenization with replace_number set to False
2021-05-16 21:14:11,253 INFO current processing data/used/train_set/IRB201901754_NOTE_609932_1.txt ...
2021-05-16 21:14:11,258 WARNING ' 336.0' cannot be parsed by current rule.
2021-05-16 21:14:11,259 WARNING '2. Pulmonary' cannot be parsed by current rule.
2021-05-16 21:14:11,260 WARNING '3. Peripheral' cannot be parsed by current rule.
2021-05-16 21:14:11,260 WARNING '4. Essential' cannot be parsed by current rule.
2021-05-16 21:14:11,261 WARNING '5. Closed' cannot be pars

2021-05-16 21:14:12,784 INFO current processing data/used/train_set/note_111176_0.txt ...
2021-05-16 21:14:12,793 WARNING '1.73M2' cannot be parsed by current rule.
2021-05-16 21:14:12,816 INFO process note_111176_0 file
2021-05-16 21:14:12,817 WARNING ('wine', 'Alcohol_type', (3283, 3287)) offset is overlapped with previous entity; current tok not overlap
2021-05-16 21:14:12,868 INFO sentence boundary detection class initiated.
2021-05-16 21:14:12,869 INFO word level tokenization with replace_number set to False
2021-05-16 21:14:12,869 INFO current processing data/used/train_set/note_188677_2.txt ...
2021-05-16 21:14:12,889 INFO process note_188677_2 file
2021-05-16 21:14:12,938 INFO sentence boundary detection class initiated.
2021-05-16 21:14:12,938 INFO word level tokenization with replace_number set to False
2021-05-16 21:14:12,939 INFO current processing data/used/train_set/IRB201901754_NOTE_503633_0.txt ...
2021-05-16 21:14:12,967 INFO process IRB201901754_NOTE_503633_0 file
202

2021-05-16 21:14:14,790 INFO sentence boundary detection class initiated.
2021-05-16 21:14:14,791 INFO word level tokenization with replace_number set to False
2021-05-16 21:14:14,792 INFO current processing data/used/train_set/note_123150_0.txt ...
2021-05-16 21:14:14,871 INFO process note_123150_0 file
2021-05-16 21:14:14,930 INFO sentence boundary detection class initiated.
2021-05-16 21:14:14,931 INFO word level tokenization with replace_number set to False
2021-05-16 21:14:14,932 INFO current processing data/used/train_set/IRB201901754_NOTE_661687_0.txt ...
2021-05-16 21:14:14,962 INFO process IRB201901754_NOTE_661687_0 file
2021-05-16 21:14:15,014 INFO sentence boundary detection class initiated.
2021-05-16 21:14:15,014 INFO word level tokenization with replace_number set to False
2021-05-16 21:14:15,015 INFO current processing data/used/train_set/IRB201901754_NOTE_1204956_0.txt ...
2021-05-16 21:14:15,048 INFO process IRB201901754_NOTE_1204956_0 file
2021-05-16 21:14:15,049 WARN

2021-05-16 21:14:16,616 INFO sentence boundary detection class initiated.
2021-05-16 21:14:16,617 INFO word level tokenization with replace_number set to False
2021-05-16 21:14:16,617 INFO current processing data/used/train_set/IRB201901754_NOTE_1228367_0.txt ...
2021-05-16 21:14:16,642 INFO process IRB201901754_NOTE_1228367_0 file
2021-05-16 21:14:16,643 WARNING ('beer', 'Alcohol_type', (2608, 2612)) offset is overlapped with previous entity; current tok not overlap
2021-05-16 21:14:16,693 INFO sentence boundary detection class initiated.
2021-05-16 21:14:16,694 INFO word level tokenization with replace_number set to False
2021-05-16 21:14:16,694 INFO current processing data/used/train_set/IRB201901754_NOTE_710085_0.txt ...
2021-05-16 21:14:16,757 INFO process IRB201901754_NOTE_710085_0 file
2021-05-16 21:14:16,806 INFO sentence boundary detection class initiated.
2021-05-16 21:14:16,807 INFO word level tokenization with replace_number set to False
2021-05-16 21:14:16,808 INFO current

In [11]:
# train dev split
file_ids = list(file_ids)
train_ids, dev_ids = train_test_split(file_ids, train_size=0.9, random_state=13, shuffle=True)
len(train_ids), len(dev_ids)

(452, 51)

In [12]:
# merge previous bio results into train.txt and dev.txt
import fileinput

# train
with open("./bio_final/train.txt", "w") as f:
    for fid in train_ids:
        f.writelines(fileinput.input(output_root / (fid + ".bio.txt")))
    fileinput.close()
        
# dev
with open("./bio_final/dev.txt", "w") as f:
    for fid in dev_ids:
        f.writelines(fileinput.input(output_root / (fid + ".bio.txt")))
    fileinput.close()
    
# now you can check your ./2018n2c2/bio, there should be two new files train.txt and dev.txt
! ls -l ./bio_final/

total 30740
-rw-rw-r--. 1 zehao.yu zehao.yu  2665160 May 16 21:14 dev.txt
drwxrwxr-x. 2 zehao.yu zehao.yu    24576 May 16 21:14 trains
-rw-rw-r--. 1 zehao.yu zehao.yu 28770166 May 16 21:14 train.txt


In [13]:
#train ner
! export CUDA_VISIBLE_DEVICES=1  # indicate no GPU; if you have two GPUs then the IDs for them will be 0 and 1
#MKL_THREADING_LAYER=GNU
! MKL_THREADING_LAYER=GNU python ./ClinicalTransformerNER/src/run_transformer_ner.py \
      --model_type bert \
      --pretrained_model bert-base-uncased \
      --data_dir ./bio_final \
      --new_model_dir ./model/SDOH_bert_final \
      --overwrite_model_dir \
      --max_seq_length 128 \
      --data_has_offset_information \
      --save_model_core \
      --do_train \
      --model_selection_scoring strict-f_score-1 \
      --do_lower_case \
      --train_batch_size 8 \
      --train_steps 1000 \
      --learning_rate 1e-5 \
      --num_train_epochs 20 \
      --gradient_accumulation_steps 1 \
      --do_warmup \
      --seed 13 \
      --warmup_ratio 0.1 \
      --max_num_checkpoints 3 \
      --log_file ./log_bert.txt \
      --progress_bar \
      --early_stop 3 

Downloading: 100%|███████████████████████████| 232k/232k [00:00<00:00, 2.17MB/s]
Downloading: 100%|██████████████████████████████| 570/570 [00:00<00:00, 451kB/s]
Downloading: 100%|███████████████████████████| 440M/440M [02:19<00:00, 3.15MB/s]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertNerModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertNerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertNerModel from the checkpoint of a model that you expect to be exactly identical 

In [1]:
cat ./log_bert.txt

2021-05-16 21:17:16 - Transformer_NER - INFO - Task will use cuda device: GPU_0.
2021-05-16 21:17:16 - Transformer_NER - WARNING - set tokenizer as bert-base-uncased
2021-05-16 21:17:16 - Transformer_NER - WARNING - set config as bert-base-uncased
2021-05-16 21:17:17 - Transformer_NER - INFO - Training/evaluation parameters: {'model_type': 'bert', 'pretrained_model': 'bert-base-uncased', 'config_name': 'bert-base-uncased', 'tokenizer_name': 'bert-base-uncased', 'data_dir': './bio_final', 'data_has_offset_information': True, 'new_model_dir': './model/SDOH_bert_final', 'save_model_core': True, 'predict_output_file': None, 'overwrite_model_dir': True, 'seed': 13, 'max_seq_length': 128, 'do_train': True, 'model_selection_scoring': 'strict-f_score-1', 'do_predict': False, 'use_crf': False, 'do_lower_case': True, 'train_batch_size': 8, 'eval_batch_size': 8, 'train_steps': 1000, 'learning_rate': 1e-05, 'num_train_epochs': 20.0, 'gradient_accumulation_steps': 1, 'do_warmup': True, 'warmup_rati

Physical Exam ED Triage Vitals BP 11 / 06 / 15 1713 91 / 63 mmHg Pulse 11 / 06 / 15 1713 135 Resp 11 / 06 / 15 1713 14 Temp 11 / 06 / 15 1713 37.3 °C ( 99.2 °F ) Temp src - - Height 11 / 06 / 15 1713 1.854 m ( 6 ' 1 " " ) Weight 11 / 06 / 15 1713 68.04 kg ( 150 lb ) SpO 2 11 / 06 / 15 1713 100 % BMI ( Calculated ) 11 / 06 / 15 1713 19.83 BP 103 / 65 mmHg | Pulse 129 | Temp ( Src ) 37.3 °C ( 99.2 °F ) | Resp 15 | Ht 1.854 m ( 6 ' 1 " " ) | Wt 68.04 kg ( 150 lb ) | BMI 19.79 kg / m 2 | SpO 2 96 % Physical Exam Differential DDx : * * * Is this an Emergent Medical Condition ? Yes - Severe Pain / Acute Onset of Symptons 409.901 FS 641.19 FS 627.732 ( 16 ) FS ED Course Procedures MDM ED Re - Evaluation ED Disposition ED Disposition : No ED Disposition Set ED Clinical Impression ED Clinical Impression None ED Patient Status Patient Status : The patient condition on admission / discharge / transfer is : { SH ED UF PATIENT CONDITION : 1602057101 } ED Medical Evaluation Initiated Medical Evaluat

5 / 27 / 16 Townsend , Petra Elke , MD Allergies : Allergies Allergen Reactions • Percocet [ Oxycodone - Acetaminophen ] Shortness Of Breath and Itching • Penicillins Rash Family History : Family History Problem Relation Age of Onset • Other Mother diverticulosis , DVT • Heart Disease Mother • Asthma Mother • High Blood Pressure Mother • High Cholesterol Mother • Other Son seizures • Other Daughter AML kidney Social History : Social History Socioeconomic History • Marital status : Divorced Spouse name : Not on file • Number of children : Not on file • Years of education : Not on file • Highest education level : Not on file Occupational History • Occupation : retired • Occupation : social security admin ( claims ) Social Needs • Financial resource strain : Not on file • Food insecurity : Worry : Not on file Inability : Not on file • Transportation needs : Medical : Not on file Non - medical : Not on file Tobacco Use • Smoking status : Current Every Day Smoker Packs / day : 1.00 Years : 

General Assessment : * * * ANTICIPATORY GUIDANCE REVIEWED : Yes No Comments Injury Prevention : Water / drowning , home safety ( poisons ) , forward facing car seat ( seatcheck . org ) , bike helmet , supervise outside , sun protection x Sleep : patterns , consistent routines , night waking , snoring ? x Toddler development : model appropriate language , daily reading , listen and respond to child , feelings and emotions , plays with other children x Oral Health : No bottles , brush teeth 2 x / day , establish dental home x Activity / Media : Limit TV to < 1 - 2 hrs / days , discuss alternatives ( reading , games , singing ) , encourage physical activity x Toilet training : signs of readiness , plan for frequent toilet breaks , personal hygiene x Family : family planning , domestic violence , time for self partner , community activities x Passive Smoke x PAH x ROR x PROCEDURES / SCREENING : Yes No Comments Hb x Lead Risk Factors x Lead obtained : no TB Risk Factors x Cholesterol ( if h

The patient has : Health Maintenance Reminders Due Completion Dates TDAP / TD VACCINE ( Q 10 YR , 18 - 99 + YRS ) 9 / 6 / 1969 - - - PAP SMEAR ( Q 2 YR , WOMEN 21 - 65 YRS ) 9 / 6 / 1972 - - - COLONOSCOPY ( Q 10 YR , 50 - 75 YRS ) 9 / 6 / 2001 - - - ZOSTER VACCINE ( 60 + YRS ) 9 / 6 / 2011 - - - MAMMOGRAM DISCUSSION ( WOMEN 40 - 75 YRS ) 8 / 6 / 2015 8 / 6 / 2014 , 7 / 22 / 2013 , 4 / 25 / 2012 INFLUENZA VACCINE ( QSEASON , 18 - 99 + YRS ) 9 / 1 / 2015 - - - LIPID PROFILE ( Q 5 YR , WOMEN 45 + YRS ) 12 / 9 / 2019 12 / 9 / 2014 , 11 / 27 / 2012 , 8 / 28 / 2012 No changes made .

2021-05-19 22:04:13 - Transformer_NER - WARNING - 
Sentence after tokenization is too long (expect less than 126 but got 145).
You have to increase max_seq_length to make sure every sentence after tokenization is not longer than the max_seq_length.
Or you can further split you long sentences into shorter ones. We will defaultly segment the sentence to two or more seqs with length as (~max_seq_len * n + rest).
Th

Allergies Allergen Reactions • No Known Drug Allergy No Known Drug Allergy • Strawberry Itching and Swelling ROS : Constitutional : Negative Eyes : Negative Ears , nose , mouth , throat , and face : Negative Respiratory : Negative Cardiovascular : Negative Gastrointestinal : Negative Genitourinary : Negative Integument / breast : Negative Hematologic / lymphatic : Negative Musculoskeletal : Negative Neurological : Negative Behavioral / Psych : Negative Endocrine : negative Allergic / Immunologic : negative Constitutional : Denies Fatigue , Denies Fever , Denies Chills , Denies Weight change , Denies headaches , Eyes : No visual disturbances , ENT : Denies Ear disease , Denies nose bleeds , Denies Dry mouth , Denies mouth sores , Cardiovascular : No Murmur , Denies heart palpitations , Denies chest pain , Respiratory : No SOB , Denies Wheezing , Denies Cough , Denies DOE , Musculoskeletal : Denies joint pain , Back pain , or Difficulty walking , Gastrointestinal : Denies Nausea , Denies

Lab Review : Results for orders placed or performed in visit on 07 / 31 / 19 POCT Urinalysis w / o Microscopy auto Result Value Ref Range Clarity , UA Color - Ur Glucose - Ur neg Bilirubin - Ur small Ketones trace Spec Grav 1.020 Blood , UA neg pH 7.0 Protein - Ur 100 Urobilinogen - Ur 2.0 Nitrite - Ur neg Leukocytes neg Appearance , Fluid Lot Number Expiration Date * Note : Due to a large number of results and / or encounters for the requested time period , some results have not been displayed .

2021-05-19 22:04:30 - Transformer_NER - WARNING - 
Sentence after tokenization is too long (expect less than 126 but got 146).
You have to increase max_seq_length to make sure every sentence after tokenization is not longer than the max_seq_length.
Or you can further split you long sentences into shorter ones. We will defaultly segment the sentence to two or more seqs with length as (~max_seq_len * n + rest).
The extra long sentence: 
Basic Metabolic labs : Lab Results Component Value Date CR

In [5]:
! ls -l ./model/SDOH_bert_final

total 1712048
-rw-rw-r--. 1 zehao.yu zehao.yu        17 May 20 00:09 added_tokens.json
-rw-rw-r--. 1 zehao.yu zehao.yu       101 May 19 22:04 base_model_name.txt
-rw-rw-r--. 1 zehao.yu zehao.yu 438282889 May 19 23:02 checkpoint_23999.bin
-rw-rw-r--. 1 zehao.yu zehao.yu 438282889 May 19 23:07 checkpoint_25999.bin
-rw-rw-r--. 1 zehao.yu zehao.yu 438282889 May 19 23:28 checkpoint_34999.bin
-rw-rw-r--. 1 zehao.yu zehao.yu      6356 May 20 00:09 config.json
-rw-rw-r--. 1 zehao.yu zehao.yu      1806 May 19 22:04 label2idx.json
-rw-rw-r--. 1 zehao.yu zehao.yu 438018551 May 19 23:28 pytorch_model.bin
-rw-rw-r--. 1 zehao.yu zehao.yu       112 May 20 00:09 special_tokens_map.json
-rw-rw-r--. 1 zehao.yu zehao.yu       300 May 20 00:09 tokenizer_config.json
-rw-rw-r--. 1 zehao.yu zehao.yu    231508 May 20 00:09 vocab.txt


In [6]:
test_root = Path('./data/used/test_set')
file_ids = set()
enss = []

for fn in Path(test_root).glob("*.ann"):
    file_ids.add(fn.stem)
    _, ens, _ = read_annotation_brat(fn)
    #print( _)
    enss.extend(ens)
print("training files: ", len(file_ids), list(file_ids)[:5])
print("total training eneitites: ", len(enss))
print("Entities distribution by types:\n", "\n".join([str(c) for c in Counter([each[1] for each in enss]).most_common()]))

training files:  126 ['note_75096_0', 'IRB201901754_NOTE_1070244_0', 'IRB201901754_NOTE_1185859_2', 'IRB201901754_NOTE_662843_0', 'IRB201901754_NOTE_504433_0']
total training eneitites:  2687
Entities distribution by types:
 ('Substance_use_status', 464)
('Sdoh_status', 330)
('Gender', 179)
('Smoking_type', 146)
('Education', 122)
('Alcohol_use', 115)
('Living_supply', 114)
('Marital_status', 113)
('Drug_use', 111)
('Sex_act', 95)
('Occupation', 93)
('Social_cohesion', 91)
('Abuse', 88)
('Smoking_freq_ppd', 71)
('Alcohol_freq', 62)
('Tobacco_use', 56)
('Smoking_freq_sy', 52)
('Physical_act', 48)
('Sdoh_freq', 43)
('Transportation', 39)
('Smoking_freq_qy', 35)
('Partner', 34)
('Living_Condition', 28)
('Race', 20)
('Financial_constrain', 19)
('Alcohol_other', 17)
('Smoking_freq_py', 14)
('SDoH_ICD', 13)
('Employment_status', 12)
('Drug_type', 12)
('Ethnicity', 8)
('Alcohol_type', 8)
('Protection', 8)
('Smoking_freq_other', 7)
('Language', 6)
('Gender_status', 5)
('Drug_freq', 3)
('Drug_o

In [8]:
# generate bio
test_root = Path(test_root)
test_bio = "./bio_final/test"
output_root = Path(test_bio)
output_root.mkdir(parents=True, exist_ok=True)

for fn in test_root.glob("*.txt"):
    txt_fn = fn
    bio_fn = output_root / (fn.stem + ".bio.txt")
    
    txt, sents = pre_processing(txt_fn, deid_pattern=MIMICIII_PATTERN)
    nsents, sent_bound = generate_BIO(sents, [], file_id=fn, no_overlap=False)
    
    BIOdata_to_file(bio_fn, nsents)

2021-05-20 00:14:51,348 INFO sentence boundary detection class initiated.
2021-05-20 00:14:51,349 INFO word level tokenization with replace_number set to False
2021-05-20 00:14:51,349 INFO current processing data/used/test_set/note_75096_0.txt ...
2021-05-20 00:14:51,369 INFO process data/used/test_set/note_75096_0.txt file
2021-05-20 00:14:51,418 INFO sentence boundary detection class initiated.
2021-05-20 00:14:51,419 INFO word level tokenization with replace_number set to False
2021-05-20 00:14:51,419 INFO current processing data/used/test_set/IRB201901754_NOTE_230291_0.txt ...
2021-05-20 00:14:51,456 INFO process data/used/test_set/IRB201901754_NOTE_230291_0.txt file
2021-05-20 00:14:51,505 INFO sentence boundary detection class initiated.
2021-05-20 00:14:51,506 INFO word level tokenization with replace_number set to False
2021-05-20 00:14:51,507 INFO current processing data/used/test_set/IRB201901754_NOTE_1117092_0.txt ...
2021-05-20 00:14:51,543 INFO process data/used/test_set/I

2021-05-20 00:14:53,403 INFO word level tokenization with replace_number set to False
2021-05-20 00:14:53,404 INFO current processing data/used/test_set/IRB201901754_NOTE_1240986_2.txt ...
2021-05-20 00:14:53,425 INFO process data/used/test_set/IRB201901754_NOTE_1240986_2.txt file
2021-05-20 00:14:53,476 INFO sentence boundary detection class initiated.
2021-05-20 00:14:53,477 INFO word level tokenization with replace_number set to False
2021-05-20 00:14:53,478 INFO current processing data/used/test_set/IRB201901754_NOTE_83731_0.txt ...
2021-05-20 00:14:53,559 INFO process data/used/test_set/IRB201901754_NOTE_83731_0.txt file
2021-05-20 00:14:53,609 INFO sentence boundary detection class initiated.
2021-05-20 00:14:53,610 INFO word level tokenization with replace_number set to False
2021-05-20 00:14:53,611 INFO current processing data/used/test_set/IRB201901754_NOTE_1070244_0.txt ...
2021-05-20 00:14:53,633 INFO process data/used/test_set/IRB201901754_NOTE_1070244_0.txt file
2021-05-20

2021-05-20 00:14:55,413 INFO process data/used/test_set/IRB201901754_NOTE_505372_10.txt file
2021-05-20 00:14:55,460 INFO sentence boundary detection class initiated.
2021-05-20 00:14:55,461 INFO word level tokenization with replace_number set to False
2021-05-20 00:14:55,461 INFO current processing data/used/test_set/note_293938_1.txt ...
2021-05-20 00:14:55,484 WARNING '1.73M2' cannot be parsed by current rule.
2021-05-20 00:14:55,506 INFO process data/used/test_set/note_293938_1.txt file
2021-05-20 00:14:55,556 INFO sentence boundary detection class initiated.
2021-05-20 00:14:55,557 INFO word level tokenization with replace_number set to False
2021-05-20 00:14:55,557 INFO current processing data/used/test_set/note_15031_0.txt ...
2021-05-20 00:14:55,623 INFO process data/used/test_set/note_15031_0.txt file
2021-05-20 00:14:55,682 INFO sentence boundary detection class initiated.
2021-05-20 00:14:55,683 INFO word level tokenization with replace_number set to False
2021-05-20 00:14:5

2021-05-20 00:14:57,509 INFO sentence boundary detection class initiated.
2021-05-20 00:14:57,510 INFO word level tokenization with replace_number set to False
2021-05-20 00:14:57,511 INFO current processing data/used/test_set/note_108380_1.txt ...
2021-05-20 00:14:57,532 INFO process data/used/test_set/note_108380_1.txt file
2021-05-20 00:14:57,581 INFO sentence boundary detection class initiated.
2021-05-20 00:14:57,584 INFO word level tokenization with replace_number set to False
2021-05-20 00:14:57,586 INFO current processing data/used/test_set/note_235978_1.txt ...
2021-05-20 00:14:57,640 INFO process data/used/test_set/note_235978_1.txt file
2021-05-20 00:14:57,693 INFO sentence boundary detection class initiated.
2021-05-20 00:14:57,693 INFO word level tokenization with replace_number set to False
2021-05-20 00:14:57,694 INFO current processing data/used/test_set/note_121879_0.txt ...
2021-05-20 00:14:57,722 INFO process data/used/test_set/note_121879_0.txt file
2021-05-20 00:14

2021-05-20 00:14:59,818 INFO sentence boundary detection class initiated.
2021-05-20 00:14:59,819 INFO word level tokenization with replace_number set to False
2021-05-20 00:14:59,819 INFO current processing data/used/test_set/IRB201901754_NOTE_603724_0.txt ...
2021-05-20 00:14:59,844 INFO process data/used/test_set/IRB201901754_NOTE_603724_0.txt file
2021-05-20 00:14:59,893 INFO sentence boundary detection class initiated.
2021-05-20 00:14:59,894 INFO word level tokenization with replace_number set to False
2021-05-20 00:14:59,895 INFO current processing data/used/test_set/IRB201901754_NOTE_908918_0.txt ...
2021-05-20 00:14:59,951 INFO process data/used/test_set/IRB201901754_NOTE_908918_0.txt file
2021-05-20 00:15:00,029 INFO sentence boundary detection class initiated.
2021-05-20 00:15:00,030 INFO word level tokenization with replace_number set to False
2021-05-20 00:15:00,030 INFO current processing data/used/test_set/note_170557_0.txt ...
2021-05-20 00:15:00,038 WARNING '1.73M2' ca

2021-05-20 00:15:01,868 INFO current processing data/used/test_set/note_289144_10.txt ...
2021-05-20 00:15:01,907 INFO process data/used/test_set/note_289144_10.txt file
2021-05-20 00:15:01,955 INFO sentence boundary detection class initiated.
2021-05-20 00:15:01,956 INFO word level tokenization with replace_number set to False
2021-05-20 00:15:01,956 INFO current processing data/used/test_set/IRB201901754_NOTE_715448_0.txt ...
2021-05-20 00:15:01,979 WARNING '2A.Pnt' cannot be parsed by current rule.
2021-05-20 00:15:02,007 INFO process data/used/test_set/IRB201901754_NOTE_715448_0.txt file
2021-05-20 00:15:02,057 INFO sentence boundary detection class initiated.
2021-05-20 00:15:02,058 INFO word level tokenization with replace_number set to False
2021-05-20 00:15:02,058 INFO current processing data/used/test_set/note_234000_0.txt ...
2021-05-20 00:15:02,106 INFO process data/used/test_set/note_234000_0.txt file
2021-05-20 00:15:02,157 INFO sentence boundary detection class initiated.

In [60]:

! export CUDA_VISIBLE_DEVICES=1

# use format 1 for BRAT, 2 for BioC, 0 as default for BIO
! MKL_THREADING_LAYER=GNU python ./ClinicalTransformerNER/src/run_transformer_batch_prediction.py \
      --model_type bert \
      --pretrained_model ./model/SDOH_bert_final \
      --raw_text_dir ./data/used/test_set/ \
      --preprocessed_text_dir ./bio_final/test/ \
      --output_dir ./res/bert_final \
      --max_seq_length 128 \
      --do_lower_case \
      --eval_batch_size 8 \
      --log_file ./log.txt\
      --do_format 1 \
      --do_copy \
      --data_has_offset_information

In [11]:
! python brat_eval.py --f1 ./data/used/test_set --f2 ./res/bert_final_formatted_output/

******************* Information Extraction Results *******************
                      ------- strict -------    ------ lenient -------
                      Prec.   Rec.    F(b=1)    Prec.   Rec.    F(b=1)
                Race  1.0000  0.9500  0.9744    1.0000  0.9500  0.9744
               Abuse  0.9512  0.8864  0.9176    0.9756  0.9091  0.9412
              Gender  0.9337  0.9441  0.9389    0.9337  0.9441  0.9389
             Sex_act  0.9684  0.9684  0.9684    0.9789  0.9789  0.9789
             Partner  0.9706  0.9706  0.9706    1.0000  1.0000  1.0000
            Sdoh_icd  0.7273  0.6154  0.6667    0.8182  0.6923  0.7500
            Drug_use  0.9902  0.9099  0.9484    0.9902  0.9099  0.9484
            Language  1.0000  1.0000  1.0000    1.0000  1.0000  1.0000
           Sdoh_freq  0.9556  1.0000  0.9773    0.9556  1.0000  0.9773
           Drug_type  0.8889  0.6667  0.7619    0.8889  0.6667  0.7619
           Education  0.9593  0.9672  0.9633    0.9675  0.9754  0.9714
      

# other model training code

In [10]:
python ./ClinicalTransformerNER/src/run_transformer_ner.py \
      --model_type roberta \
      --pretrained_model roberta-base \
      --data_dir ./bio_final \
      --new_model_dir ./model/SDOH_robert_final \
      --overwrite_model_dir \
      --max_seq_length 128 \
      --data_has_offset_information \
      --save_model_core \
      --do_train \
      --model_selection_scoring strict-f_score-1 \
      --do_lower_case \
      --train_batch_size 8 \
      --train_steps 1000 \
      --learning_rate 1e-5 \
      --num_train_epochs 20 \
      --gradient_accumulation_steps 1 \
      --do_warmup \
      --seed 13 \
      --warmup_ratio 0.1 \
      --max_num_checkpoints 3 \
      --log_file ./log.txt \
      --progress_bar \
      --early_stop 3 

SyntaxError: invalid syntax (<ipython-input-10-2bd20f78dff9>, line 1)

In [ ]:
python ./ClinicalTransformerNER/src/run_transformer_batch_prediction.py \
      --model_type roberta \
      --pretrained_model ./model/SDOH_robert_final \
      --raw_text_dir ./data/used/test_set/ \
      --preprocessed_text_dir ./bio_final/test/ \
      --output_dir ./res/roberta_final \
      --max_seq_length 128 \
      --do_lower_case \
      --eval_batch_size 8 \
      --log_file ./log.txt\
      --do_format 1 \
      --do_copy \
      --data_has_offset_information

In [12]:
! python brat_eval.py --f1 ./data/used/test_set --f2 ./res/roberta_final_formatted_output/

******************* Information Extraction Results *******************
                      ------- strict -------    ------ lenient -------
                      Prec.   Rec.    F(b=1)    Prec.   Rec.    F(b=1)
                Race  0.9474  0.9000  0.9231    0.9474  0.9000  0.9231
               Abuse  0.9176  0.8864  0.9017    0.9412  0.9091  0.9249
              Gender  0.8850  0.9888  0.9340    0.8850  0.9888  0.9340
             Sex_act  0.9020  0.9684  0.9340    0.9314  1.0000  0.9645
             Partner  0.9091  0.8824  0.8955    1.0000  0.9706  0.9851
            Sdoh_icd  0.6667  0.6154  0.6400    0.7500  0.6923  0.7200
            Drug_use  0.9636  0.9550  0.9593    0.9727  0.9640  0.9683
            Language  0.8571  1.0000  0.9231    0.8571  1.0000  0.9231
           Sdoh_freq  0.9773  1.0000  0.9885    0.9773  1.0000  0.9885
           Education  0.9174  0.9098  0.9136    0.9583  0.9426  0.9504
           Ethnicity  1.0000  1.0000  1.0000    1.0000  1.0000  1.0000
      

In [ ]:
python ./ClinicalTransformerNER/src/run_transformer_ner.py \
      --model_type roberta \
      --pretrained_model /home/zehao.yu/workspace/py3/dr_ann/pre_train_model/mimiciii_roberta_10e_128b \
      --data_dir ./bio_final \
      --new_model_dir ./model/SDOH_roberta_mimic_final \
      --overwrite_model_dir \
      --max_seq_length 128 \
      --data_has_offset_information \
      --save_model_core \
      --do_train \
      --model_selection_scoring strict-f_score-1 \
      --do_lower_case \
      --train_batch_size 8 \
      --train_steps 1000 \
      --learning_rate 1e-5 \
      --num_train_epochs 20 \
      --gradient_accumulation_steps 1 \
      --do_warmup \
      --seed 13 \
      --warmup_ratio 0.1 \
      --max_num_checkpoints 3 \
      --log_file ./log.txt \
      --progress_bar \
      --early_stop 3 

In [ ]:
python ./ClinicalTransformerNER/src/run_transformer_batch_prediction.py \
      --model_type roberta \
      --pretrained_model ./model/SDOH_roberta_mimic_final \
      --raw_text_dir ./data/used/test_set/ \
      --preprocessed_text_dir ./bio_final/test/ \
      --output_dir ./res/roberta_mimic_final \
      --max_seq_length 128 \
      --do_lower_case \
      --eval_batch_size 8 \
      --log_file ./log.txt\
      --do_format 1 \
      --do_copy \
      --data_has_offset_information

In [13]:
! python brat_eval.py --f1 ./data/used/test_set --f2 ./res/roberta_mimic_final_formatted_output/

******************* Information Extraction Results *******************
                      ------- strict -------    ------ lenient -------
                      Prec.   Rec.    F(b=1)    Prec.   Rec.    F(b=1)
                Race  0.9474  0.9000  0.9231    0.9474  0.9000  0.9231
               Abuse  0.8387  0.8864  0.8619    0.8710  0.9205  0.8950
              Gender  0.9189  0.9497  0.9341    0.9189  0.9497  0.9341
             Sex_act  0.9684  0.9684  0.9684    0.9895  0.9895  0.9895
             Partner  0.9118  0.9118  0.9118    1.0000  1.0000  1.0000
            Sdoh_icd  0.6154  0.6154  0.6154    0.7692  0.7692  0.7692
            Language  0.8571  1.0000  0.9231    0.8571  1.0000  0.9231
            Drug_use  0.9541  0.9369  0.9455    0.9725  0.9550  0.9636
           Drug_freq  0.7500  1.0000  0.8571    0.7500  1.0000  0.8571
           Sdoh_freq  1.0000  1.0000  1.0000    1.0000  1.0000  1.0000
           Ethnicity  0.8889  1.0000  0.9412    0.8889  1.0000  0.9412
      

In [ ]:
python ./ClinicalTransformerNER/src/run_transformer_ner.py \
      --model_type bert \
      --pretrained_model /home/zehao.yu/workspace/py3/dr_ann/pre_train_model/mimiciii_bert_10e_128b \
      --data_dir ./bio_final \
      --new_model_dir ./model/SDOH_bert_mimic_final \
      --overwrite_model_dir \
      --max_seq_length 128 \
      --data_has_offset_information \
      --save_model_core \
      --do_train \
      --model_selection_scoring strict-f_score-1 \
      --do_lower_case \
      --train_batch_size 8 \
      --train_steps 1000 \
      --learning_rate 1e-5 \
      --num_train_epochs 20 \
      --gradient_accumulation_steps 1 \
      --do_warmup \
      --seed 13 \
      --warmup_ratio 0.1 \
      --max_num_checkpoints 3 \
      --log_file ./log.txt \
      --progress_bar \
      --early_stop 3 

In [ ]:
python ./ClinicalTransformerNER/src/run_transformer_batch_prediction.py \
      --model_type bert \
      --pretrained_model ./model/SDOH_bert_mimic_final \
      --raw_text_dir ./data/used/test_set/ \
      --preprocessed_text_dir ./bio_final/test/ \
      --output_dir ./res/bert_mimic_final \
      --max_seq_length 128 \
      --do_lower_case \
      --eval_batch_size 8 \
      --log_file ./log.txt\
      --do_format 1 \
      --do_copy \
      --data_has_offset_information

In [14]:
! python brat_eval.py --f1 ./data/used/test_set --f2 ./res/bert_mimic_final_formatted_output/

ERROR: None of the files match.
